Select the correct kernel.

In [1]:
import sys

In [6]:
print(sys.executable)
# Should give : c:\Users\Sebastien\Desktop\LLM and GenAI\Smart_Resume_to_Job_Matcher\.venv\Scripts\python.exe

c:\Users\Sebastien\Desktop\LLM and GenAI\Smart_Resume_to_Job_Matcher\.venv\Scripts\python.exe


In [8]:
!ollama list

NAME                      ID              SIZE      MODIFIED    
mxbai-embed-large:335m    468836162de7    669 MB    2 weeks ago    
qwen3-vl:2b               0635d9d857d4    1.9 GB    2 weeks ago    
smollm2:135m              9077fe9d2ae1    270 MB    2 weeks ago    
llama3:latest             365c0bd3c000    4.7 GB    4 weeks ago    


In [12]:
from langchain_community.llms import Ollama

In [13]:
from langchain_community.embeddings import OllamaEmbeddings

In [14]:
from langgraph.graph import StateGraph

In [15]:
import streamlit
import fastapi

In [ ]:
emb = OllamaEmbeddings(model="nomic-embed-text")
print(len(emb.embed_query("test embedding")))